In [17]:
#import library yang dibutuhkan
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re
import os

In [18]:
#memanggil data yang akan diproses (harus format csv)

def load_data():
    data = pd.read_csv('Data/crawling_data_ruu.csv')
    return data

df = load_data()
df.head()

,Datetime,Tweet Id,Location,Username,Text
0,2022-12-13 23:24:59+00:00,1.602807e+18,"Batam, Indonesia",gokepricom,b'Dispar Kepri Jamin RUU KUHP yang Baru Tak Ga...
1,2022-12-13 23:23:28+00:00,1.602806e+18,"Bali, Indonesia",MenSang3,"b'Ini RUU KUHP maksudnya...pacaran di hotel, M..."
2,2022-12-13 23:21:42+00:00,1.602806e+18,NaN,UmmuZaki30,"b'""Pasal Karet"" RUU KUHP AntiDemokrasi: Menyer..."
3,2022-12-13 22:13:03+00:00,1.602789e+18,Indonesia,kompasiana,b'Kohabitasi antara Penolakan dan Penerimaan s...
4,2022-12-13 22:01:28+00:00,1.602786e+18,NaN,DemokratProbo1,"b'jangan diam, perjuangkan hak rakyat salam RU..."


In [19]:
#menghapus username dalam tweet
def remove_pattern(Text, pattern):
    r = re.findall(pattern, Text)
    for i in r:
        Text = re.sub(i, '', Text)
    return Text
df['remove_user'] = np.vectorize(remove_pattern)(df['Text'], "@[\w]*")
df.head()

,Datetime,Tweet Id,Location,Username,Text,remove_user
0,2022-12-13 23:24:59+00:00,1.602807e+18,"Batam, Indonesia",gokepricom,b'Dispar Kepri Jamin RUU KUHP yang Baru Tak Ga...,b'Dispar Kepri Jamin RUU KUHP yang Baru Tak Ga...
1,2022-12-13 23:23:28+00:00,1.602806e+18,"Bali, Indonesia",MenSang3,"b'Ini RUU KUHP maksudnya...pacaran di hotel, M...","b'Ini RUU KUHP maksudnya...pacaran di hotel, M..."
2,2022-12-13 23:21:42+00:00,1.602806e+18,NaN,UmmuZaki30,"b'""Pasal Karet"" RUU KUHP AntiDemokrasi: Menyer...","b'""Pasal Karet"" RUU KUHP AntiDemokrasi: Menyer..."
3,2022-12-13 22:13:03+00:00,1.602789e+18,Indonesia,kompasiana,b'Kohabitasi antara Penolakan dan Penerimaan s...,b'Kohabitasi antara Penolakan dan Penerimaan s...
4,2022-12-13 22:01:28+00:00,1.602786e+18,NaN,DemokratProbo1,"b'jangan diam, perjuangkan hak rakyat salam RU...","b'jangan diam, perjuangkan hak rakyat salam RU..."


In [20]:
def normalization(Text):
    Text = re.sub(r'\$\w*', '', Text)
    Text = re.sub(r'^rt[\s]+', '', Text)
    Text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', ' ', Text)
    Text = re.sub('&quot;'," ", Text)
    Text = re.sub(r"\d+", " ", str(Text))
    Text = re.sub(r"\b[a-zA-Z]\b", "", str(Text))
    Text = re.sub(r"[^\w\s]", " ", str(Text))
    Text = re.sub(r'(.)\1+', r'\1\1', Text)
    Text = re.sub(r"\s+", " ", str(Text))
    Text = re.sub(r'#', '', Text)
    Text = re.sub(r'[^a-zA-z0-9]', ' ', str(Text))
    Text = re.sub(r'\b\w{1,2}\b', '', Text)
    Text = re.sub(r'\s\s+', ' ', Text)
    Text = re.sub(r'^RT[\s]+', '', Text)
    Text = re.sub(r'^b[\s]+', '', Text)
    Text = re.sub(r'^link[\s]+', '', Text)
    return Text

df['normalisasi'] = df['remove_user'].apply(normalization)
df.head()

,Datetime,Tweet Id,Location,Username,Text,remove_user,normalisasi
0,2022-12-13 23:24:59+00:00,1.602807e+18,"Batam, Indonesia",gokepricom,b'Dispar Kepri Jamin RUU KUHP yang Baru Tak Ga...,b'Dispar Kepri Jamin RUU KUHP yang Baru Tak Ga...,Dispar Kepri Jamin RUU KUHP yang Baru Tak Gan...
1,2022-12-13 23:23:28+00:00,1.602806e+18,"Bali, Indonesia",MenSang3,"b'Ini RUU KUHP maksudnya...pacaran di hotel, M...","b'Ini RUU KUHP maksudnya...pacaran di hotel, M...",Ini RUU KUHP maksudnya pacaran hotel polisi g...
2,2022-12-13 23:21:42+00:00,1.602806e+18,NaN,UmmuZaki30,"b'""Pasal Karet"" RUU KUHP AntiDemokrasi: Menyer...","b'""Pasal Karet"" RUU KUHP AntiDemokrasi: Menyer...",Pasal Karet RUU KUHP AntiDemokrasi Menyeret R...
3,2022-12-13 22:13:03+00:00,1.602789e+18,Indonesia,kompasiana,b'Kohabitasi antara Penolakan dan Penerimaan s...,b'Kohabitasi antara Penolakan dan Penerimaan s...,Kohabitasi antara Penolakan dan Penerimaan se...
4,2022-12-13 22:01:28+00:00,1.602786e+18,NaN,DemokratProbo1,"b'jangan diam, perjuangkan hak rakyat salam RU...","b'jangan diam, perjuangkan hak rakyat salam RU...",jangan diam perjuangkan hak rakyat salam RUU ...


In [21]:
#menghapus duplikat
df.drop_duplicates(subset ="normalisasi", keep = 'first', inplace = True)

In [27]:
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)

In [29]:
df['case_folding'] = df['normalisasi'].str.lower()
df.head()

,Datetime,Tweet Id,Location,Username,Text,remove_user,normalisasi,case_folding,tokenisasi,stopword_removal
0,2022-12-13 23:24:59+00:00,1.602807e+18,"Batam, Indonesia",gokepricom,b'Dispar Kepri Jamin RUU KUHP yang Baru Tak Ga...,b'Dispar Kepri Jamin RUU KUHP yang Baru Tak Ga...,Dispar Kepri Jamin RUU KUHP yang Baru Tak Gan...,dispar kepri jamin ruu kuhp yang baru tak gan...,"[dispar, kepri, jamin, ruu, kuhp, yang, baru, ...","[dispar, kepri, jamin, ruu, kuhp, ganggu, pari..."
1,2022-12-13 23:23:28+00:00,1.602806e+18,"Bali, Indonesia",MenSang3,"b'Ini RUU KUHP maksudnya...pacaran di hotel, M...","b'Ini RUU KUHP maksudnya...pacaran di hotel, M...",Ini RUU KUHP maksudnya pacaran hotel polisi g...,ini ruu kuhp maksudnya pacaran hotel polisi g...,"[ini, ruu, kuhp, maksudnya, pacaran, hotel, po...","[ruu, kuhp, maksudnya, pacaran, hotel, polisi,..."
2,2022-12-13 23:21:42+00:00,1.602806e+18,NaN,UmmuZaki30,"b'""Pasal Karet"" RUU KUHP AntiDemokrasi: Menyer...","b'""Pasal Karet"" RUU KUHP AntiDemokrasi: Menyer...",Pasal Karet RUU KUHP AntiDemokrasi Menyeret R...,pasal karet ruu kuhp antidemokrasi menyeret r...,"[pasal, karet, ruu, kuhp, antidemokrasi, menye...","[pasal, karet, ruu, kuhp, antidemokrasi, menye..."
3,2022-12-13 22:13:03+00:00,1.602789e+18,Indonesia,kompasiana,b'Kohabitasi antara Penolakan dan Penerimaan s...,b'Kohabitasi antara Penolakan dan Penerimaan s...,Kohabitasi antara Penolakan dan Penerimaan se...,kohabitasi antara penolakan dan penerimaan se...,"[kohabitasi, antara, penolakan, dan, penerimaa...","[kohabitasi, penolakan, penerimaan, dampak, pe..."
4,2022-12-13 22:01:28+00:00,1.602786e+18,NaN,DemokratProbo1,"b'jangan diam, perjuangkan hak rakyat salam RU...","b'jangan diam, perjuangkan hak rakyat salam RU...",jangan diam perjuangkan hak rakyat salam RUU ...,jangan diam perjuangkan hak rakyat salam ruu ...,"[jangan, diam, perjuangkan, hak, rakyat, salam...","[diam, perjuangkan, hak, rakyat, salam, ruu, k..."


In [30]:
from nltk.tokenize import word_tokenize 

def word_tokenize_wrapper(Text):
    return word_tokenize(Text)

df['tokenisasi'] = df['normalisasi'].apply(lambda x: word_tokenize_wrapper(x.lower()))
df.head()

,Datetime,Tweet Id,Location,Username,Text,remove_user,normalisasi,case_folding,tokenisasi,stopword_removal
0,2022-12-13 23:24:59+00:00,1.602807e+18,"Batam, Indonesia",gokepricom,b'Dispar Kepri Jamin RUU KUHP yang Baru Tak Ga...,b'Dispar Kepri Jamin RUU KUHP yang Baru Tak Ga...,Dispar Kepri Jamin RUU KUHP yang Baru Tak Gan...,dispar kepri jamin ruu kuhp yang baru tak gan...,"[dispar, kepri, jamin, ruu, kuhp, yang, baru, ...","[dispar, kepri, jamin, ruu, kuhp, ganggu, pari..."
1,2022-12-13 23:23:28+00:00,1.602806e+18,"Bali, Indonesia",MenSang3,"b'Ini RUU KUHP maksudnya...pacaran di hotel, M...","b'Ini RUU KUHP maksudnya...pacaran di hotel, M...",Ini RUU KUHP maksudnya pacaran hotel polisi g...,ini ruu kuhp maksudnya pacaran hotel polisi g...,"[ini, ruu, kuhp, maksudnya, pacaran, hotel, po...","[ruu, kuhp, maksudnya, pacaran, hotel, polisi,..."
2,2022-12-13 23:21:42+00:00,1.602806e+18,NaN,UmmuZaki30,"b'""Pasal Karet"" RUU KUHP AntiDemokrasi: Menyer...","b'""Pasal Karet"" RUU KUHP AntiDemokrasi: Menyer...",Pasal Karet RUU KUHP AntiDemokrasi Menyeret R...,pasal karet ruu kuhp antidemokrasi menyeret r...,"[pasal, karet, ruu, kuhp, antidemokrasi, menye...","[pasal, karet, ruu, kuhp, antidemokrasi, menye..."
3,2022-12-13 22:13:03+00:00,1.602789e+18,Indonesia,kompasiana,b'Kohabitasi antara Penolakan dan Penerimaan s...,b'Kohabitasi antara Penolakan dan Penerimaan s...,Kohabitasi antara Penolakan dan Penerimaan se...,kohabitasi antara penolakan dan penerimaan se...,"[kohabitasi, antara, penolakan, dan, penerimaa...","[kohabitasi, penolakan, penerimaan, dampak, pe..."
4,2022-12-13 22:01:28+00:00,1.602786e+18,NaN,DemokratProbo1,"b'jangan diam, perjuangkan hak rakyat salam RU...","b'jangan diam, perjuangkan hak rakyat salam RU...",jangan diam perjuangkan hak rakyat salam RUU ...,jangan diam perjuangkan hak rakyat salam ruu ...,"[jangan, diam, perjuangkan, hak, rakyat, salam...","[diam, perjuangkan, hak, rakyat, salam, ruu, k..."


In [32]:
from nltk.corpus import stopwords

list_stopwords = stopwords.words('indonesian')

list_stopwords.extend(['yg', 'dg', 'rt', 'dgn', 'ny', 'd', 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&', 'yah', 'no', 'je', 'om', 'pru', 'sch',
                       'injirrr', 'ah', 'oena', 'bu', 'eh'])

list_stopwords = set(list_stopwords)

def stopwords_removal(Text):
  return [word for word in Text if word not in list_stopwords]

df['stopword_removal'] = df['tokenisasi'].apply(stopwords_removal)
df.head()

,Datetime,Tweet Id,Location,Username,Text,remove_user,normalisasi,case_folding,tokenisasi,stopword_removal
0,2022-12-13 23:24:59+00:00,1.602807e+18,"Batam, Indonesia",gokepricom,b'Dispar Kepri Jamin RUU KUHP yang Baru Tak Ga...,b'Dispar Kepri Jamin RUU KUHP yang Baru Tak Ga...,Dispar Kepri Jamin RUU KUHP yang Baru Tak Gan...,dispar kepri jamin ruu kuhp yang baru tak gan...,"[dispar, kepri, jamin, ruu, kuhp, yang, baru, ...","[dispar, kepri, jamin, ruu, kuhp, ganggu, pari..."
1,2022-12-13 23:23:28+00:00,1.602806e+18,"Bali, Indonesia",MenSang3,"b'Ini RUU KUHP maksudnya...pacaran di hotel, M...","b'Ini RUU KUHP maksudnya...pacaran di hotel, M...",Ini RUU KUHP maksudnya pacaran hotel polisi g...,ini ruu kuhp maksudnya pacaran hotel polisi g...,"[ini, ruu, kuhp, maksudnya, pacaran, hotel, po...","[ruu, kuhp, maksudnya, pacaran, hotel, polisi,..."
2,2022-12-13 23:21:42+00:00,1.602806e+18,NaN,UmmuZaki30,"b'""Pasal Karet"" RUU KUHP AntiDemokrasi: Menyer...","b'""Pasal Karet"" RUU KUHP AntiDemokrasi: Menyer...",Pasal Karet RUU KUHP AntiDemokrasi Menyeret R...,pasal karet ruu kuhp antidemokrasi menyeret r...,"[pasal, karet, ruu, kuhp, antidemokrasi, menye...","[pasal, karet, ruu, kuhp, antidemokrasi, menye..."
3,2022-12-13 22:13:03+00:00,1.602789e+18,Indonesia,kompasiana,b'Kohabitasi antara Penolakan dan Penerimaan s...,b'Kohabitasi antara Penolakan dan Penerimaan s...,Kohabitasi antara Penolakan dan Penerimaan se...,kohabitasi antara penolakan dan penerimaan se...,"[kohabitasi, antara, penolakan, dan, penerimaa...","[kohabitasi, penolakan, penerimaan, dampak, pe..."
4,2022-12-13 22:01:28+00:00,1.602786e+18,NaN,DemokratProbo1,"b'jangan diam, perjuangkan hak rakyat salam RU...","b'jangan diam, perjuangkan hak rakyat salam RU...",jangan diam perjuangkan hak rakyat salam RUU ...,jangan diam perjuangkan hak rakyat salam ruu ...,"[jangan, diam, perjuangkan, hak, rakyat, salam...","[diam, perjuangkan, hak, rakyat, salam, ruu, k..."


In [33]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import swifter

#buat stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#stemmed wrapper
def stemmed_wrapper(term):
  return stemmer.stem(term)

term_dict = {}

for Text in df['stopword_removal']:
  for term in Text:
    if term not in term_dict:
      term_dict[term] = ' '

print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")

#memmulai stemming
def apply_stemmed_term(Text):
  return [term_dict[term] for term in Text]

df['stemming'] = df['stopword_removal'].swifter.apply(apply_stemmed_term)
df.head()

6541
------------------------
dispar : dispar
kepri : kepri
jamin : jamin
ruu : ruu
kuhp : kuhp
ganggu : ganggu
pariwisata : pariwisata
maksudnya : maksud
pacaran : pacar
hotel : hotel
polisi : polisi
gerebek : gerebek
ortunya : ortunya
lapor : lapor
anaknya : anak
dilaporin : dilaporin
pasal : pasal
karet : karet
antidemokrasi : antidemokrasi
menyeret : seret
rakyat : rakyat
penjara : penjara
menjamin : jamin
kenyamanan : nyaman
penguasa : kuasa
kohabitasi : kohabitasi
penolakan : tolak
penerimaan : terima
dampak : dampak
pengesahan : kesah
hlkompasiana : hlkompasiana
diam : diam
perjuangkan : juang
hak : hak
salam : salam
janban : janban
mati : mati
terjerat : jerat
aturan : atur
pemerintah : perintah
ahy : ahy
ahypimpinperubahan : ahypimpinperubahan
demokrat : demokrat
demokratjatim : demokratjatim
demokratharapanrakyat : demokratharapanrakyat
emilsangdemokrat : emilsangdemokrat
disorot : sorot
pengurangan : kurang
tahanan : tahan
koruptor : koruptor
dendanya : denda
tujuan : tuju
k

Pandas Apply:   0%|          | 0/3553 [00:00<?, ?it/s]

,Datetime,Tweet Id,Location,Username,Text,remove_user,normalisasi,case_folding,tokenisasi,stopword_removal,stemming
0,2022-12-13 23:24:59+00:00,1.602807e+18,"Batam, Indonesia",gokepricom,b'Dispar Kepri Jamin RUU KUHP yang Baru Tak Ga...,b'Dispar Kepri Jamin RUU KUHP yang Baru Tak Ga...,Dispar Kepri Jamin RUU KUHP yang Baru Tak Gan...,dispar kepri jamin ruu kuhp yang baru tak gan...,"[dispar, kepri, jamin, ruu, kuhp, yang, baru, ...","[dispar, kepri, jamin, ruu, kuhp, ganggu, pari...","[dispar, kepri, jamin, ruu, kuhp, ganggu, pari..."
1,2022-12-13 23:23:28+00:00,1.602806e+18,"Bali, Indonesia",MenSang3,"b'Ini RUU KUHP maksudnya...pacaran di hotel, M...","b'Ini RUU KUHP maksudnya...pacaran di hotel, M...",Ini RUU KUHP maksudnya pacaran hotel polisi g...,ini ruu kuhp maksudnya pacaran hotel polisi g...,"[ini, ruu, kuhp, maksudnya, pacaran, hotel, po...","[ruu, kuhp, maksudnya, pacaran, hotel, polisi,...","[ruu, kuhp, maksud, pacar, hotel, polisi, gere..."
2,2022-12-13 23:21:42+00:00,1.602806e+18,NaN,UmmuZaki30,"b'""Pasal Karet"" RUU KUHP AntiDemokrasi: Menyer...","b'""Pasal Karet"" RUU KUHP AntiDemokrasi: Menyer...",Pasal Karet RUU KUHP AntiDemokrasi Menyeret R...,pasal karet ruu kuhp antidemokrasi menyeret r...,"[pasal, karet, ruu, kuhp, antidemokrasi, menye...","[pasal, karet, ruu, kuhp, antidemokrasi, menye...","[pasal, karet, ruu, kuhp, antidemokrasi, seret..."
3,2022-12-13 22:13:03+00:00,1.602789e+18,Indonesia,kompasiana,b'Kohabitasi antara Penolakan dan Penerimaan s...,b'Kohabitasi antara Penolakan dan Penerimaan s...,Kohabitasi antara Penolakan dan Penerimaan se...,kohabitasi antara penolakan dan penerimaan se...,"[kohabitasi, antara, penolakan, dan, penerimaa...","[kohabitasi, penolakan, penerimaan, dampak, pe...","[kohabitasi, tolak, terima, dampak, kesah, ruu..."
4,2022-12-13 22:01:28+00:00,1.602786e+18,NaN,DemokratProbo1,"b'jangan diam, perjuangkan hak rakyat salam RU...","b'jangan diam, perjuangkan hak rakyat salam RU...",jangan diam perjuangkan hak rakyat salam RUU ...,jangan diam perjuangkan hak rakyat salam ruu ...,"[jangan, diam, perjuangkan, hak, rakyat, salam...","[diam, perjuangkan, hak, rakyat, salam, ruu, k...","[diam, juang, hak, rakyat, salam, ruu, kuhp, j..."


In [34]:
stemming = df[['stemming']]

def fit_stemming(text):
    text = np.array(text)
    text = ' '.join(text)
    
    return text

df['stemming'] = df['stemming'].apply(lambda x: fit_stemming(x))
df.head()

,Datetime,Tweet Id,Location,Username,Text,remove_user,normalisasi,case_folding,tokenisasi,stopword_removal,stemming
0,2022-12-13 23:24:59+00:00,1.602807e+18,"Batam, Indonesia",gokepricom,b'Dispar Kepri Jamin RUU KUHP yang Baru Tak Ga...,b'Dispar Kepri Jamin RUU KUHP yang Baru Tak Ga...,Dispar Kepri Jamin RUU KUHP yang Baru Tak Gan...,dispar kepri jamin ruu kuhp yang baru tak gan...,"[dispar, kepri, jamin, ruu, kuhp, yang, baru, ...","[dispar, kepri, jamin, ruu, kuhp, ganggu, pari...",dispar kepri jamin ruu kuhp ganggu pariwisata
1,2022-12-13 23:23:28+00:00,1.602806e+18,"Bali, Indonesia",MenSang3,"b'Ini RUU KUHP maksudnya...pacaran di hotel, M...","b'Ini RUU KUHP maksudnya...pacaran di hotel, M...",Ini RUU KUHP maksudnya pacaran hotel polisi g...,ini ruu kuhp maksudnya pacaran hotel polisi g...,"[ini, ruu, kuhp, maksudnya, pacaran, hotel, po...","[ruu, kuhp, maksudnya, pacaran, hotel, polisi,...",ruu kuhp maksud pacar hotel polisi gerebek ort...
2,2022-12-13 23:21:42+00:00,1.602806e+18,NaN,UmmuZaki30,"b'""Pasal Karet"" RUU KUHP AntiDemokrasi: Menyer...","b'""Pasal Karet"" RUU KUHP AntiDemokrasi: Menyer...",Pasal Karet RUU KUHP AntiDemokrasi Menyeret R...,pasal karet ruu kuhp antidemokrasi menyeret r...,"[pasal, karet, ruu, kuhp, antidemokrasi, menye...","[pasal, karet, ruu, kuhp, antidemokrasi, menye...",pasal karet ruu kuhp antidemokrasi seret rakya...
3,2022-12-13 22:13:03+00:00,1.602789e+18,Indonesia,kompasiana,b'Kohabitasi antara Penolakan dan Penerimaan s...,b'Kohabitasi antara Penolakan dan Penerimaan s...,Kohabitasi antara Penolakan dan Penerimaan se...,kohabitasi antara penolakan dan penerimaan se...,"[kohabitasi, antara, penolakan, dan, penerimaa...","[kohabitasi, penolakan, penerimaan, dampak, pe...",kohabitasi tolak terima dampak kesah ruu kuhp ...
4,2022-12-13 22:01:28+00:00,1.602786e+18,NaN,DemokratProbo1,"b'jangan diam, perjuangkan hak rakyat salam RU...","b'jangan diam, perjuangkan hak rakyat salam RU...",jangan diam perjuangkan hak rakyat salam RUU ...,jangan diam perjuangkan hak rakyat salam ruu ...,"[jangan, diam, perjuangkan, hak, rakyat, salam...","[diam, perjuangkan, hak, rakyat, salam, ruu, k...",diam juang hak rakyat salam ruu kuhp janban ra...


In [35]:
#filtering duplikat
df.drop_duplicates(subset = "stemming", keep = 'first', inplace = True)
df

,Datetime,Tweet Id,Location,Username,Text,remove_user,normalisasi,case_folding,tokenisasi,stopword_removal,stemming
0,2022-12-13 23:24:59+00:00,1.602807e+18,"Batam, Indonesia",gokepricom,b'Dispar Kepri Jamin RUU KUHP yang Baru Tak Ga...,b'Dispar Kepri Jamin RUU KUHP yang Baru Tak Ga...,Dispar Kepri Jamin RUU KUHP yang Baru Tak Gan...,dispar kepri jamin ruu kuhp yang baru tak gan...,"[dispar, kepri, jamin, ruu, kuhp, yang, baru, ...","[dispar, kepri, jamin, ruu, kuhp, ganggu, pari...",dispar kepri jamin ruu kuhp ganggu pariwisata
1,2022-12-13 23:23:28+00:00,1.602806e+18,"Bali, Indonesia",MenSang3,"b'Ini RUU KUHP maksudnya...pacaran di hotel, M...","b'Ini RUU KUHP maksudnya...pacaran di hotel, M...",Ini RUU KUHP maksudnya pacaran hotel polisi g...,ini ruu kuhp maksudnya pacaran hotel polisi g...,"[ini, ruu, kuhp, maksudnya, pacaran, hotel, po...","[ruu, kuhp, maksudnya, pacaran, hotel, polisi,...",ruu kuhp maksud pacar hotel polisi gerebek ort...
2,2022-12-13 23:21:42+00:00,1.602806e+18,NaN,UmmuZaki30,"b'""Pasal Karet"" RUU KUHP AntiDemokrasi: Menyer...","b'""Pasal Karet"" RUU KUHP AntiDemokrasi: Menyer...",Pasal Karet RUU KUHP AntiDemokrasi Menyeret R...,pasal karet ruu kuhp antidemokrasi menyeret r...,"[pasal, karet, ruu, kuhp, antidemokrasi, menye...","[pasal, karet, ruu, kuhp, antidemokrasi, menye...",pasal karet ruu kuhp antidemokrasi seret rakya...
3,2022-12-13 22:13:03+00:00,1.602789e+18,Indonesia,kompasiana,b'Kohabitasi antara Penolakan dan Penerimaan s...,b'Kohabitasi antara Penolakan dan Penerimaan s...,Kohabitasi antara Penolakan dan Penerimaan se...,kohabitasi antara penolakan dan penerimaan se...,"[kohabitasi, antara, penolakan, dan, penerimaa...","[kohabitasi, penolakan, penerimaan, dampak, pe...",kohabitasi tolak terima dampak kesah ruu kuhp ...
4,2022-12-13 22:01:28+00:00,1.602786e+18,NaN,DemokratProbo1,"b'jangan diam, perjuangkan hak rakyat salam RU...","b'jangan diam, perjuangkan hak rakyat salam RU...",jangan diam perjuangkan hak rakyat salam RUU ...,jangan diam perjuangkan hak rakyat salam ruu ...,"[jangan, diam, perjuangkan, hak, rakyat, salam...","[diam, perjuangkan, hak, rakyat, salam, ruu, k...",diam juang hak rakyat salam ruu kuhp janban ra...
...,...,...,...,...,...,...,...,...,...,...,...
6052,2022-08-01 02:24:59+00:00,1.553930e+18,Cagliari,Duke_of_Nowhere,b'@henrysubiakto Emang itu hak setiap warga ne...,b' Emang itu hak setiap warga negara. Tapi dik...,Emang itu hak setiap warga negara Tapi dikrit...,emang itu hak setiap warga negara tapi dikrit...,"[emang, itu, hak, setiap, warga, negara, tapi,...","[emang, hak, warga, negara, dikritiknya, jamin...",emang hak warga negara kritik jamin kkn mata g...
6061,2022-08-01 02:05:35+00:00,1.553925e+18,"Padang Timur, Indonesia",premanlnsaf,b'Dukungan Serta Testimoni Tokoh/Kelompok Masy...,b'Dukungan Serta Testimoni Tokoh/Kelompok Masy...,Dukungan Serta Testimoni Tokoh Kelompok Masya...,dukungan serta testimoni tokoh kelompok masya...,"[dukungan, serta, testimoni, tokoh, kelompok, ...","[dukungan, testimoni, tokoh, kelompok, masyara...",dukung testimoni tokoh kelompok masyarakat per...
6065,2022-08-01 02:01:00+00:00,1.553924e+18,"Maluku, Indonesia",CahyaniIka3,b'IKN Potret Kemajuan\nKekhawatiran soal pengh...,b'IKN Potret Kemajuan\nKekhawatiran soal pengh...,IKN Potret Kemajuan nKekhawatiran soal penghi...,ikn potret kemajuan nkekhawatiran soal penghi...,"[ikn, potret, kemajuan, nkekhawatiran, soal, p...","[ikn, potret, kemajuan, nkekhawatiran, penghin...",ikn potret maju nkekhawatiran hina presiden ru...
6066,2022-08-01 01:35:00+00:00,1.553917e+18,"Sumatera Utara, Indonesia",AnjaniAneira,b'RUU KUHP penting untuk segera disahkan. IKN ...,b'RUU KUHP penting untuk segera disahkan. IKN ...,RUU KUHP penting untuk segera disahkan IKN Po...,ruu kuhp penting untuk segera disahkan ikn po...,"[ruu, kuhp, penting, untuk, segera, disahkan, ...","[ruu, kuhp, disahkan, ikn, potret, kemajuan]",ruu kuhp sah ikn potret maju


In [36]:
#savecsv
df.to_csv('preprocessing_ruu.csv',encoding='utf8', index=False)